In [1]:
import datetime
import os
import seaborn as sns
import sys
import time

%matplotlib inline

In [2]:
SCRIPTS_DIR = '../app'

sys.path.insert(1, SCRIPTS_DIR)

## Objective: New features to MA Trader 
### [1] Feat: MaxDrawdown as metric and NumExecution for information display [100%]
### [2] Feat: new trading strategy based upon double moving averages [100%]
### [3] Feat: new trading strategy w/ MACD [10%]

#### Prepare Data

In [3]:
# config

SECONDS_IN_ONE_DAY = 86400

STRATEGIES = ['MA-SELVES', 'DOUBLE-MA']

# simulation configuration
TOL_PCTS = [0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
BUY_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]
SELL_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]
MA_LENGTHS = [3, 5, 10, 20, 30]
BUY_STAS = ('by_percentage')
SELL_STAS = ('by_percentage')

# currencies (digital + real-world)
CURS = ['BTC', 'ETH', 'LTC', 'BCH', 'ETC']
FIAT = ['USD']

# signals
NO_ACTION_SIGNAL = 'NO ACTION'
BUY_SIGNAL = 'BUY'
SELL_SIGNAL = 'SELL'

# epilson
EP_COIN = 10e-3
EP_CASH = 5

In [4]:
from trader_driver import TraderDriver
from util import load_csv

name,l = load_csv(csv_fn='../tests/fixtures/BTC_HISTORY.csv')

l = l[-365:]

In [5]:
init_amount = 3000
cur_coin = 5
mode = 'normal'

t_driver = TraderDriver(
    name=name,
    init_amount=init_amount,
    overall_stats=STRATEGIES,
    cur_coin=cur_coin,
    tol_pcts=TOL_PCTS,
    ma_lengths=MA_LENGTHS,
    buy_pcts=BUY_PCTS,
    sell_pcts=SELL_PCTS,
    buy_stas=BUY_STAS,
    sell_stas=SELL_STAS,
    mode=mode
)

t_driver.feed_data(data_stream=l)

info = t_driver.best_trader_info
# best trader
best_t = t_driver.traders[info['trader_index']]
# for a new price, find the trade signal
signal = best_t.trade_signal

signal

--> executed=feed_data using time=9.63 seconds.


{'action': 'NO ACTION', 'buy_percentage': 0.7, 'sell_percentage': 0.7}

In [6]:
best_t.all_history[:2]

[{'action': 'NO ACTION',
  'price': 7175.6674765579,
  'date': '2020-04-08',
  'coin': 5,
  'cash': 3000,
  'portfolio': 38878.3373827895},
 {'action': 'NO ACTION',
  'price': 7367.293398450201,
  'date': '2020-04-09',
  'coin': 5,
  'cash': 3000,
  'portfolio': 39836.466992251}]

In [7]:
best_t.all_history_trade_only

[]

In [8]:
best_t.max_drawdown, best_t.num_transaction, best_t.high_strategy

(0.1813, 0, 'MA-SELVES')

In [9]:
info

{'buy_pct': 0.7,
 'sell_pct': 0.7,
 'tol_pct': 0.5,
 'buy': 'by_percentage',
 'sell': 'by_percentage',
 'init_value': 38878.3373827895,
 'max_final_value': 68514.672,
 'rate_of_return': '76.228%',
 'baseline_rate_of_return': '76.228%',
 'coin_rate_of_return': '82.602%',
 'trader_index': 249}

In [10]:
best_t.rate_of_return, best_t.baseline_rate_of_return, best_t.coin_rate_of_return

(76.228, 76.228, 82.602)

In [11]:
best_t.all_history[0], best_t.all_history[-1]

({'action': 'NO ACTION',
  'price': 7175.6674765579,
  'date': '2020-04-08',
  'coin': 5,
  'cash': 3000,
  'portfolio': 38878.3373827895},
 {'action': 'NO ACTION',
  'price': 13102.9344615485,
  'date': '2020-10-22',
  'coin': 5,
  'cash': 3000,
  'portfolio': 68514.6723077425})